In [2]:
import torchvision.models as models
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import torch.nn as nn

In [1]:
!pip install gdown

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device:
    print('Model is running on GPU:', device)
else:
    print('Model is running on CPU')

class InpaintingDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_transform=None, mask_transform=None, subfolder_limit=10):
        self.image_dir = image_dir
        self.mask_dir = mask_dir

        # Always initialize image_list and mask_list to ensure they are defined
        self.image_list = []
        self.mask_list = []

        if subfolder_limit > 0:
            self.subfolder_limit = subfolder_limit  # New parameter to limit subfolders

            # Populate the lists with files from the first N subfolders
            image_subfolders = sorted([os.path.join(image_dir, name) for name in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, name))])[:subfolder_limit]

            for subfolder in image_subfolders:
                for file in os.listdir(subfolder):
                    if os.path.splitext(file)[1].lower() in ['.jpg', '.png', '.jpeg']:
                        self.image_list.append(os.path.join(subfolder, file))
        else:
            self.image_list = [os.path.join(dp, f) for dp, dn, filenames in os.walk(image_dir) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg', '.png', '.jpeg']]

        # Populate mask_list with all files from the mask directory
        self.mask_list = [os.path.join(dp, f) for dp, dn, filenames in os.walk(mask_dir) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg', '.png', '.jpeg']]

        self.image_transform = image_transform
        self.mask_transform = mask_transform

    def __len__(self):
        return len(self.image_list)
    
    def dilate_mask(self, mask, dilation_kernel_size=3):
        kernel = np.ones((dilation_kernel_size, dilation_kernel_size), np.uint8)
        dilated_mask = cv2.dilate(mask.numpy(), kernel, iterations=1)
        return torch.from_numpy(dilated_mask)

    def create_weight_map(self, mask, dilated_mask, border_weight=2.0):
        border = dilated_mask - mask
        weight_map = torch.ones_like(mask)
        weight_map[border == 1] = border_weight
        return weight_map  # Add this line


    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        # Randomly select a mask
        mask_path = random.choice(self.mask_list)

        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('1')

        if self.image_transform:
            image = self.image_transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)

        # Ensure mask is a binary tensor with the same size as image in the channel dimension
        mask = mask.expand_as(image)

        masked_image = image * (1 - mask)
        
        mask = (mask > 0).float()

        masked_image = image * (1 - mask)

        dilated_mask = self.dilate_mask(mask)
        weight_map = self.create_weight_map(mask, dilated_mask)
        
        
        
        weighted_masked_image = masked_image * weight_map

        return {
            'ground_truth': image, 
            'weighted_masked_image': weighted_masked_image, 
            'mask': dilated_mask
            }

image_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
])

# Define your mask transformations including random rotation, flip, and dilation
mask_transform = transforms.Compose([
    transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.NEAREST),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
])

# Create dataset instances
train_dataset = InpaintingDataset(
    # image_dir='E:/1807033/Dataset/TEMPTRAINIMG',
    # mask_dir='E:/1807033/Dataset/TEMPTRAINMASK',
    image_dir='/kaggle/input/places2-mit-dataset/train_256_places365standard/data_256/a/',
    mask_dir='/kaggle/input/training-mask',
    image_transform=image_transform,
    mask_transform=mask_transform,
    subfolder_limit=10  # Only include the first 10 subfolders
)

val_dataset = InpaintingDataset(
    image_dir='/kaggle/input/validation-image',
    mask_dir='/kaggle/input/validation-mask',
    # image_dir='E:/1807033/Dataset/TEMPVALIMG',
    # mask_dir='E:/1807033/Dataset/TEMPVALMASK',
    image_transform=image_transform,
    mask_transform=mask_transform,
    subfolder_limit=0
)

from PIL import Image
import os


train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
class VGG16FeatureExtractor(nn.Module):
    def __init__(self):
        super(VGG16FeatureExtractor, self).__init__()
        vgg16 = models.vgg16(pretrained=True).features
        self.features = nn.Sequential(*list(vgg16.children())[:23])  # Adjust based on the layers you need
        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.features(x)

class ContentStyleLoss(nn.Module):
    def __init__(self):
        super(ContentStyleLoss, self).__init__()
        self.feature_extractor = VGG16FeatureExtractor()

    def compute_gram_matrix(self, input):
        a, b, c, d = input.size()  # a=batch size(=1)
        features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL
        G = torch.mm(features, features.t())  # compute the gram product
        return G.div(a * b * c * d)

    def forward(self, generated, target):
        gen_features = self.feature_extractor(generated)
        target_features = self.feature_extractor(target)
        content_loss = F.mse_loss(gen_features, target_features)

        # Compute style loss
        gen_gram = self.compute_gram_matrix(gen_features)
        target_gram = self.compute_gram_matrix(target_features)
        style_loss = F.mse_loss(gen_gram, target_gram)

        return content_loss, style_loss

In [ ]:

class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        vgg19 = models.vgg19(pretrained=True).features
        self.vgg19 = nn.Sequential(*list(vgg19.children())[:36]).eval()  # Up to the second conv layer in the 5th block
        for param in self.vgg19.parameters():
            param.requires_grad = False

    def forward(self, inpainted_image, target_image):
        perception_loss = nn.MSELoss()
        return perception_loss(self.vgg19(inpainted_image), self.vgg19(target_image))


In [ ]:

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # Simple discriminator model
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, padding=0),  # Adjust the kernel size/padding for final size
            nn.Sigmoid()
        )
        
    def forward(self, img):
        validity = self.model(img)
        return validity

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetDown(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, kernel_size=4, stride=2, padding=1, bias=False)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_size))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class UNetUp(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_size, out_size, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_size),
            nn.ReLU(inplace=True)
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)
        return x

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.down1 = UNetDown(3, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512, dropout=0.5)
        self.down5 = UNetDown(512, 512, dropout=0.5)
        self.down6 = UNetDown(512, 512, dropout=0.5)
        self.down7 = UNetDown(512, 512, dropout=0.5)
        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)

        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 512, dropout=0.5)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128)
        self.up7 = UNetUp(256, 64)

        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x, mask):
        # Encoder
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)

        # Decoder
        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)

        inpainted = super().forward(u7)
        
        # Blend the inpainted output with the original image outside the masked region
        # This assumes mask is 1 for regions to inpaint and 0 elsewhere
        output = (1 - mask) * image + mask * inpainted
        return self.final(u7)

In [ ]:
generator = UNet()
discriminator_d1 = DiscriminatorD1()
vgg16_feature_extractor = VGG16FeatureExtractor()  # Used within ContentStyleLoss
content_style_loss = ContentStyleLoss()
perceptual_loss = PerceptualLoss()  # Optional based on your preference

# Optimizers
optimizer_g = optim.Adam(generator.parameters(), lr=3e-4)
optimizer_d1 = optim.Adam(discriminator_d1.parameters(), lr=3e-4)

# Learning rate scheduler for decay
scheduler_g = optim.lr_scheduler.StepLR(optimizer_g, step_size=50, gamma=0.5)
scheduler_d1 = optim.lr_scheduler.StepLR(optimizer_d1, step_size=50, gamma=0.5)

from PIL import Image
import os


train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Training Loop
num_epochs = 100
alpha = 0.1  # Weight for content loss
beta = 0.2   # Weight for style loss

for epoch in range(num_epochs):
    for i, batch in enumerate(train_dataloader):
        if batch is None:
            continue
        real_images = batch['ground_truth'].to(device)
        masked_images = batch['weighted_masked_image'].to(device)
        masks = batch['mask'].to(device)
        # Generate fake images
        fake_imgs = generator(real_images, masks)

        # ---------------------
        #  Train Discriminator D1
        # ---------------------
        optimizer_d1.zero_grad()
        
        # Calculate loss for real and fake images
        real_loss = criterion_gan(discriminator_d1(real_imgs), torch.ones_like(real_imgs))
        fake_loss = criterion_gan(discriminator_d1(fake_imgs.detach()), torch.zeros_like(fake_imgs))
        d1_loss = (real_loss + fake_loss) / 2

        d1_loss.backward()
        optimizer_d1.step()

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_g.zero_grad()

        # Adversarial loss
        g_loss = criterion_gan(discriminator_d1(fake_imgs), torch.ones_like(fake_imgs))
        
        # Content and style loss
        content_loss, style_loss = content_style_loss(fake_imgs, real_imgs)
        total_loss = g_loss + alpha * content_loss + beta * style_loss

        total_loss.backward()
        optimizer_g.step()

    # Update learning rate
    scheduler_g.step()
    scheduler_d1.step()

    print(f"Epoch {epoch}/{num_epochs} - D1 Loss: {d1_loss.item()} - G Loss: {total_loss.item()}")